This script aims to generate dictionary of spatial infomation from SPL (structured panel language) file.
copyright Zac Zou

In [1]:
global result,col

def writef(fname,tail):
    f = open(fname[:-4]+".%d.dict.txt"%tail,'w')
    print('\t'.join(["position","primer","sample"]),end='\n',file=f)
    for keys,values in result.items():
        print(keys,end='\t',file=f)
        print('\t'.join(values),end='\n',file=f)
    f.close()

def spl_core(word,nlcol,nrcol):
    row = word[0]
    col = []
    n1,n2 = int(word[1]),int(word[2])
    if n1==0 or n1==1:
        if n2==0 or n2==1:
            col += list(range(1,nlcol+1,2))
        if n2==0 or n2==2:
            col += list(range(2,nlcol+1,2))
    if n1==0 or n1==2:
        if n2==0 or n2==1:
            col += list(range(13,nrcol+13,2))
        if n2==0 or n2==2:
            col += list(range(14,nrcol+13,2))
    return [[row,num] for num in col]
    
def spl_decipher(line,col):
    nlcol,nrcol = 2*len(col[0]),2*len(col[1])
    sentence = line.split()
    primer = sentence[0]
    if line[0]!='(':
        for word in sentence[1:]:
            for [row,num] in spl_core(word,nlcol,nrcol):
                result["%s%d"%(row,num)] = [primer,"%s"%col[(num-1)//12][(num-13)//2]]
    else:
        [start,end] = primer.strip("()").split('-')
        for word in sentence[1:]:
            for [row,num] in spl_core(word,nlcol,nrcol):
                for it in range(int(start[1:]),int(end[1:])+1):
                    tmp = chr(ord(row)+it-int(start[1:]))
                    result["%s%d"%(tmp,num)] = ["p%d"%it,"%s"%col[(num-1)//12][(num-13)//2]]

if __name__=="__main__":
    fname = input("please input your SPL file name: ")
    file = open(fname)
    spl = file.readlines()
    file.close()
    npanel = int(spl[0][6])
    ii = 0
    for np in range(npanel):
        result = {}
        col = [spl[ii+2].strip().split(),spl[ii+3].strip().split()]
        n_norm = int(spl[ii+4].strip().split()[1])
        for jj in range(n_norm):
            spl_decipher(spl[ii+5+jj].strip(),col)
        n_extra = int(spl[ii+5+n_norm].strip().split()[1])
        for jj in range(n_extra):
            tmp = spl[ii+6+n_norm+jj].strip().split()
            result[tmp[0]]=[tmp[1],tmp[2]]
        ii = ii+5+n_norm+n_extra
        writef(fname,np+1)
    print("Successfully converted.")

please input your SPL file name: panel_1.txt
Successfully converted.
